In [2]:
import requests
import random
import json

In [3]:
baseUrl = "http://localhost:8081/api/v1/"
keycloakUrl = "http://keycloak:8080"

# Institute invite

In [14]:
instituteUserId = "admin36@mailinator.com"
instituteRequestBody = {
    "instituteName": "te",
    "email": instituteUserId,
    "contactNumber": "!23"
}

In [15]:
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", baseUrl + "Institute/invite", headers=headers, json=instituteRequestBody)

print(response.text)
assert response.status_code == 200

{"id":"sunbird-rc.registry.invite","ver":"1.0","ets":1647339119860,"params":{"resmsgid":"","msgid":"8724eccf-98ce-4b08-9511-50827da10e27","err":"","status":"SUCCESSFUL","errmsg":""},"responseCode":"OK","result":{"Institute":{"osid":"1-c86017c8-1638-4ea2-a1eb-cc02371211f1"}}}


# get institute token

In [16]:
def getToken(userId):
    headers = {
        'content-type': 'application/x-www-form-urlencoded',
    }

    data = {
      'client_id': 'registry-frontend',
      'username': userId,
      'password': 'abcd@1234',
      'grant_type': 'password'
    }

    response = requests.post('%s/auth/realms/sunbird-rc/protocol/openid-connect/token'%(keycloakUrl), headers=headers, data=data)
#     print(response.json())
#     print(response.status_code)
    token = response.json()["access_token"]
    return token


In [17]:
token = getToken(instituteUserId)
print(token)

eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJtRTZ0WlFxSWctWHhGaVJPR21ERXZOb2ctR2Y4RklkNkNTaEJqMVQ2Qkc0In0.eyJleHAiOjE2NDczMzk3MjIsImlhdCI6MTY0NzMzOTEyMiwianRpIjoiNzVhMDViOTEtNTgyMy00NDg3LWFjZTQtZmYzYTBjOWY1MTQ4IiwiaXNzIjoiaHR0cDovL2tleWNsb2FrOjgwODAvYXV0aC9yZWFsbXMvc3VuYmlyZC1yYyIsImF1ZCI6ImFjY291bnQiLCJzdWIiOiI4OWZkNTFhOC1lMDU3LTQ5NzgtYTY4ZC1mYjZmNGVlZjE4NGQiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJyZWdpc3RyeS1mcm9udGVuZCIsInNlc3Npb25fc3RhdGUiOiI5MjE0YTI1OS1jMTUyLTRkMDktOTA1Mi1jMDcwOTY5NGJjOGQiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwczovL2xvY2FsaG9zdDo0MjAwIiwiaHR0cHM6Ly9uZGVhci54aXYuaW4iLCJodHRwOi8vbG9jYWxob3N0OjQyMDAiLCJodHRwOi8vbmRlYXIueGl2LmluIiwiaHR0cDovLzIwLjE5OC42NC4xMjgiXSwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iLCJkZWZhdWx0LXJvbGVzLW5kZWFyIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19

# get institute details

In [18]:

headers = {
  'Content-Type': 'application/json',
    'Authorization': 'bearer %s'%token,
}

response = requests.request("GET", baseUrl + "Institute", headers=headers)

print(response.text)
intituteOsid = response.json()[0]["osid"]
print(intituteOsid)
assert response.status_code == 200


[{"osUpdatedAt":"2022-03-15T10:12:00.539847Z","osCreatedAt":"2022-03-15T10:12:00.539847Z","osUpdatedBy":"","contactNumber":"!23","osCreatedBy":"","osid":"1-c86017c8-1638-4ea2-a1eb-cc02371211f1","osOwner":["89fd51a8-e057-4978-a68d-fb6f4eef184d"],"email":"admin36@mailinator.com","instituteName":"te"}]
1-c86017c8-1638-4ea2-a1eb-cc02371211f1


# Affiliate Institute

In [19]:
# add affiliation details
headers = {
    'content-type': 'application/json',
    'authorization': 'bearer %s'%token,
}

data = """{
"affiliation": [
  {
    "medium": "English",
    "board": "cbse",
    "affiliationNumber":  "123",
    "grantYear": "2000",
    "expiryYear":  "3001",
    "classes": ["Class XII"]
  }
]
}"""
# print(data)
response = requests.put('%sInstitute/%s'%(baseUrl, intituteOsid), headers=headers, data=data)
print(response)
print(response.json())
assert response.status_code == 200


# get institute details
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'bearer %s'%token,
}

response = requests.request("GET", baseUrl + "Institute", headers=headers)

# print(response.text)
intituteAffiliateOsid = response.json()[0]["affiliation"][0]["osid"]
print(intituteAffiliateOsid)
assert response.status_code == 200


<Response [200]>
{'id': 'sunbird-rc.registry.update', 'ver': '1.0', 'ets': 1647339132237, 'params': {'resmsgid': '', 'msgid': '9199f4e7-6626-467c-b2ba-e1f9546d0666', 'err': '', 'status': 'SUCCESSFUL', 'errmsg': ''}, 'responseCode': 'OK'}
1-248f58ab-77a2-4faf-afff-d32d4ff322f2


In [20]:
# raise attestation request
headers = {
    'content-type': 'application/json',
    'authorization': 'bearer %s'%token,
}

params = (
    ('send', 'true'),
)

data = {
    "entityName":"Institute",
    "entityId":"%s"%intituteOsid,
    "name":"instituteAffiliation",
    "propertiesOSID": {
        "affiliation": [intituteAffiliateOsid]
    }
}
response = requests.put(baseUrl+'send', headers=headers, params=params, json=data)
print(response.status_code)
response.json()
assert response.status_code == 200
print(response.json())
assert response.json()["params"]["status"] == "SUCCESSFUL"

200
{'id': 'sunbird-rc.registry.send', 'ver': '1.0', 'ets': 1647339133313, 'params': {'resmsgid': '', 'msgid': 'f0fef9d7-86c8-43c6-9e7c-f78591157e2b', 'err': '', 'status': 'SUCCESSFUL', 'errmsg': ''}, 'responseCode': 'OK', 'result': {'attestationOSID': '1-3c44ecbf-ae0c-45a2-a54f-8f4673424762'}}


# cbse board attest claims

In [21]:
# get claims for cbse board
# create user manually and attach board-cbse role
boardUserId = "cbse-admin"
boardToken = getToken(boardUserId)
# print(boardToken)
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'bearer %s'%boardToken,
}

response = requests.request("GET", baseUrl + "board-cbse/claims", headers=headers)
assert response.status_code == 200
print(response)

# Attesting the claim as GRANT
data = {
    "action":"GRANT_CLAIM",
}
# print(response.json())
attestingClaimId = None
for claim in response.json():
#     print(claim)
    print(claim["status"])
    if claim["status"] == "OPEN":
        attestingClaimId = claim["id"]
        print(claim["id"])
        attestResp = requests.request("POST", "%sboard-cbse/claims/%s/attest" %(baseUrl, claim["id"]), headers=headers, json=data)
        assert response.status_code == 200
        print(attestResp)
        print(attestResp.json())
        break
assert attestingClaimId != None

# intituteAffiliateOsid = response.json()[0]["affiliation"][0]["osid"]
# print(intituteAffiliateOsid)

import time
time.sleep(5)
# verify attestation status
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'bearer %s'%token,
}

response = requests.request("GET", baseUrl + "Institute", headers=headers)

# print(response.text)
for affiliation in response.json()[0]["instituteAffiliation"]:
        if affiliation["_osClaimId"] == attestingClaimId:
            assert affiliation["_osState"] == "PUBLISHED"

<Response [200]>
CLOSED
CLOSED
CLOSED
CLOSED
CLOSED
CLOSED
CLOSED
CLOSED
CLOSED
CLOSED
CLOSED
CLOSED
CLOSED
OPEN
de2a0593-e24e-4f2a-a519-386b848d20a9
<Response [200]>
{'resmsgid': '', 'msgid': 'bb33666f-e379-460c-8720-b7b7fb7735ec', 'err': '', 'status': 'SUCCESSFUL', 'errmsg': ''}


In [22]:
# get institute details
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'bearer %s'%token,
}

response = requests.request("GET", baseUrl + "Institute", headers=headers)

print(response.text)
for affiliation in response.json()[0]["instituteAffiliation"]:
        if affiliation["_osClaimId"] == attestingClaimId:
            assert affiliation["_osState"] == "PUBLISHED"

[{"osUpdatedAt":"2022-03-15T10:12:15.508669Z","osCreatedAt":"2022-03-15T10:12:00.539847Z","osUpdatedBy":"","affiliation":[{"osUpdatedAt":"2022-03-15T10:12:15.508669Z","grantYear":"2000","osUpdatedBy":"","classes":["Class XII"],"affiliationNumber":"123","expiryYear":"3001","medium":"English","osid":"1-248f58ab-77a2-4faf-afff-d32d4ff322f2","board":"cbse"}],"contactNumber":"!23","osCreatedBy":"","instituteAffiliation":[{"osUpdatedAt":"2022-03-15T10:12:15.508669Z","osUpdatedBy":"","_osState":"PUBLISHED","entityId":"1-c86017c8-1638-4ea2-a1eb-cc02371211f1","osid":"1-3c44ecbf-ae0c-45a2-a54f-8f4673424762","propertiesOSID":{"osid":"1-1d0f5d50-5411-4cb4-bb29-0eb603b91d23","osUpdatedAt":"2022-03-15T10:12:13.480929Z","affiliation":["1-248f58ab-77a2-4faf-afff-d32d4ff322f2"],"osUpdatedBy":"89fd51a8-e057-4978-a68d-fb6f4eef184d"},"propertyData":"{\"affiliation\":[{\"osUpdatedAt\":\"2022-03-15T10:12:12.461819Z\",\"grantYear\":\"2000\",\"osUpdatedBy\":\"89fd51a8-e057-4978-a68d-fb6f4eef184d\",\"classes\"

# Teacher attestations


In [107]:
# create new institutions
institute1 = "ABC1@mailinator.com"
institute2 = "XYZ1@mailinator.com"
instituteRequestBody = {
    "instituteName": "ABC",
    "email": institute1,
    "contactNumber": "ABC1"
}
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", baseUrl + "Institute/invite", headers=headers, json=instituteRequestBody)

print(response.text)
assert response.status_code == 200

instituteRequestBody = {
    "instituteName": "XYZ",
    "email": institute2,
    "contactNumber": "XYZ1"
}
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", baseUrl + "Institute/invite", headers=headers, json=instituteRequestBody)

print(response.text)
assert response.status_code == 200

{"id":"sunbird-rc.registry.invite","ver":"1.0","ets":1647346991285,"params":{"resmsgid":"","msgid":"a75d0236-cf17-498b-a41b-6ee3810c6ad5","err":"","status":"UNSUCCESSFUL","errmsg":"Username already invited / registered for Institute"},"responseCode":"OK"}


AssertionError: 

In [108]:
# create teacher
teacherUserId = "t7"
teacherRequestBody = {
  "identityDetails": {
    "fullName": "teacher1",
    "gender": "Male",
    "dob": "1995-10-28",
    "identityHolder": {
      "AADHAAR": "123412323"
    }
  },
  "contactDetails": {
    "email": "%s@g.com"%teacherUserId,
    "mobile": teacherUserId
  },
  "academicQualifications": [
    {
      "instituteOSID": "1-0e343abd-c710-45a3-a240-e5d8f30ae2d2",
      "instituteName": "ABC"
    }
  ],
  "experience": [
    {
      "instituteOSID": "1-a140c49c-3575-471c-9e6c-e079fe920349",
      "instituteName": "XYZ"
    }
  ]
}
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", baseUrl + "Teacher/invite", headers=headers, json=teacherRequestBody)

print(response.text)
assert response.status_code == 200
teacherOsid = response.json()["result"]["Teacher"]["osid"]

{"id":"sunbird-rc.registry.invite","ver":"1.0","ets":1647346998867,"params":{"resmsgid":"","msgid":"f8a9fb4f-83f4-4cc5-94bf-6fa1ad2f6006","err":"","status":"SUCCESSFUL","errmsg":""},"responseCode":"OK","result":{"Teacher":{"osid":"1-eff29030-00a3-48cf-877f-a13244ed1fc2"}}}


In [109]:
teacherToken = getToken(teacherUserId)
print(teacherToken)

eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJtRTZ0WlFxSWctWHhGaVJPR21ERXZOb2ctR2Y4RklkNkNTaEJqMVQ2Qkc0In0.eyJleHAiOjE2NDczNDc2MDAsImlhdCI6MTY0NzM0NzAwMCwianRpIjoiNmM1ZjgxZGQtMTljOS00OWQ2LTk2OWUtMjA3OTc4Mjk2MDNlIiwiaXNzIjoiaHR0cDovL2tleWNsb2FrOjgwODAvYXV0aC9yZWFsbXMvc3VuYmlyZC1yYyIsImF1ZCI6ImFjY291bnQiLCJzdWIiOiI0ZmJlODAwNS0xNjdkLTQyOTYtODdjMS0wNTNmN2ViN2I5ZTciLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJyZWdpc3RyeS1mcm9udGVuZCIsInNlc3Npb25fc3RhdGUiOiIzNWU0NjM0NS1iZTUzLTQzNTUtODlmZi1kNTNmYWEzNDI3MDQiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwczovL2xvY2FsaG9zdDo0MjAwIiwiaHR0cHM6Ly9uZGVhci54aXYuaW4iLCJodHRwOi8vbG9jYWxob3N0OjQyMDAiLCJodHRwOi8vbmRlYXIueGl2LmluIiwiaHR0cDovLzIwLjE5OC42NC4xMjgiXSwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iLCJkZWZhdWx0LXJvbGVzLW5kZWFyIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19

In [110]:
# get teacher details
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'bearer %s'%teacherToken,
}

response = requests.request("GET", baseUrl + "Teacher", headers=headers)

print(response.text)
assert response.status_code == 200
teacherEntity = response.json()[0]
print(teacherEntity["experience"][0]["osid"])
experienceOsid = teacherEntity["experience"][0]["osid"]
print(teacherEntity["academicQualifications"][0]["osid"])
qualificationOsid = teacherEntity["academicQualifications"][0]["osid"]

[{"osUpdatedAt":"2022-03-15T12:23:18.989017Z","academicQualifications":[{"osUpdatedAt":"2022-03-15T12:23:18.989017Z","osCreatedAt":"2022-03-15T12:23:18.989017Z","osUpdatedBy":"","osCreatedBy":"","instituteOSID":"1-0e343abd-c710-45a3-a240-e5d8f30ae2d2","osid":"1-ad81ff95-eb57-4b90-998c-5306a3251b7d","instituteName":"ABC"}],"osCreatedAt":"2022-03-15T12:23:18.989017Z","osUpdatedBy":"","contactDetails":{"osid":"1-ac11307d-18eb-413c-9670-18a45a7da924","osUpdatedAt":"2022-03-15T12:23:18.989017Z","osCreatedAt":"2022-03-15T12:23:18.989017Z","osUpdatedBy":"","mobile":"t7","osCreatedBy":"","email":"t7@g.com"},"osCreatedBy":"","experience":[{"osUpdatedAt":"2022-03-15T12:23:18.989017Z","osCreatedAt":"2022-03-15T12:23:18.989017Z","osUpdatedBy":"","osCreatedBy":"","osid":"1-0e225961-c800-4096-afc4-bdedbb9cbdda","instituteOSID":"1-a140c49c-3575-471c-9e6c-e079fe920349","instituteName":"XYZ"}],"osid":"1-eff29030-00a3-48cf-877f-a13244ed1fc2","osOwner":["4fbe8005-167d-4296-87c1-053f7eb7b9e7"],"identityDe

# raise teacher's education attestation

In [111]:
# raise attestation request
headers = {
    'content-type': 'application/json',
    'authorization': 'bearer %s'%teacherToken,
}

data = {
    "entityName":"Teacher",
    "entityId":"%s"%teacherOsid,
    "name":"teacherExperience",
    "propertiesOSID": {
        "experience": [experienceOsid]
    }
}
response = requests.put(baseUrl+'send', headers=headers, json=data)
print(response.status_code)
response.json()
assert response.status_code == 200
print(response.json())
assert response.json()["params"]["status"] == "SUCCESSFUL"

200
{'id': 'sunbird-rc.registry.send', 'ver': '1.0', 'ets': 1647347006133, 'params': {'resmsgid': '', 'msgid': '4ee5a6e3-da2e-4731-b86d-0d2d0c2a4f61', 'err': '', 'status': 'SUCCESSFUL', 'errmsg': ''}, 'responseCode': 'OK', 'result': {'attestationOSID': '1-be25e98c-a731-4c47-803b-32734442a389'}}


In [112]:
# get institute token
institute1Token = getToken(institute1)
print(institute1Token)
institute2Token = getToken(institute2)
print(institute2Token)

eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJtRTZ0WlFxSWctWHhGaVJPR21ERXZOb2ctR2Y4RklkNkNTaEJqMVQ2Qkc0In0.eyJleHAiOjE2NDczNDc2MDcsImlhdCI6MTY0NzM0NzAwNywianRpIjoiZGQ0MzdlYjItZTM5Mi00ZTM0LTkzNTYtYjEzM2QyNzI4NmQ4IiwiaXNzIjoiaHR0cDovL2tleWNsb2FrOjgwODAvYXV0aC9yZWFsbXMvc3VuYmlyZC1yYyIsImF1ZCI6ImFjY291bnQiLCJzdWIiOiJhNDM3ZDQwMi1lNWFiLTQ3M2QtYWZjZi04MDZhMDBjNjY1MzIiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJyZWdpc3RyeS1mcm9udGVuZCIsInNlc3Npb25fc3RhdGUiOiI4ODUyMzQzZC0yMTg2LTQ0ODEtYWRiMS04NGFjMTU5ZTY5NjAiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwczovL2xvY2FsaG9zdDo0MjAwIiwiaHR0cHM6Ly9uZGVhci54aXYuaW4iLCJodHRwOi8vbG9jYWxob3N0OjQyMDAiLCJodHRwOi8vbmRlYXIueGl2LmluIiwiaHR0cDovLzIwLjE5OC42NC4xMjgiXSwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iLCJkZWZhdWx0LXJvbGVzLW5kZWFyIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19

In [113]:
# get institute claims and grant the claim
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'bearer %s'%institute2Token,
}

response = requests.request("GET", baseUrl + "Institute/claims", headers=headers)
assert response.status_code == 200
print(response)

# Attesting the claim as GRANT
data = {
    "action":"GRANT_CLAIM",
}
print(response.json())
attestingClaimId = None
for claim in response.json():
#     print(claim)
    print(claim["status"])
    if claim["status"] == "OPEN":
        attestingClaimId = claim["id"]
        print(claim["id"])
        attestResp = requests.request("POST", "%sInstitute/claims/%s/attest" %(baseUrl, claim["id"]), headers=headers, json=data)
        assert response.status_code == 200
        print(attestResp)
        print(attestResp.json())
        break
assert attestingClaimId != None


<Response [200]>
[{'id': '8950419f-ef1a-4ebe-9ee5-6d77a4d0b2b3', 'entity': 'Teacher', 'entityId': '1-73d6607f-8158-42c6-8702-e9aca997e29d', 'propertyURI': '', 'createdAt': '2022-03-15T11:38:24.123+00:00', 'attestedOn': '2022-03-15T11:43:18.891+00:00', 'status': 'CLOSED', 'conditions': "(ATTESTOR#$.osid#.contains('1-a140c49c-3575-471c-9e6c-e079fe920349'))", 'attestorEntity': 'Institute', 'requestorName': None, 'propertyData': '{"experience":[{"osUpdatedAt":"2022-03-15T11:37:41.053052Z","osCreatedAt":"2022-03-15T11:37:41.053052Z","osUpdatedBy":"","osCreatedBy":"","osid":"1-3223d019-865d-433f-8e1c-7e72b66ff62d","instituteOSID":"1-a140c49c-3575-471c-9e6c-e079fe920349","instituteName":"XYZ"}]}', 'attestationId': '1-0d91ad9d-82ce-4702-8362-e7a31bf761de', 'attestationName': 'teacherExperience', 'closed': True}, {'id': '1060c8ae-9428-4f75-89b7-2da3fcc0ee62', 'entity': 'Teacher', 'entityId': '1-eff29030-00a3-48cf-877f-a13244ed1fc2', 'propertyURI': '', 'createdAt': '2022-03-15T12:23:26.640+00:00

In [114]:
import time
time.sleep(5)
# verify attestation status
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'bearer %s'%teacherToken,
}

response = requests.request("GET", baseUrl + "Teacher", headers=headers)

# print(response.text)
for affiliation in response.json()[0]["teacherExperience"]:
        if affiliation["_osClaimId"] == attestingClaimId:
            assert affiliation["_osState"] == "PUBLISHED"


In [115]:
# raise qualification attestation request
headers = {
    'content-type': 'application/json',
    'authorization': 'bearer %s'%teacherToken,
}

data = {
    "entityName":"Teacher",
    "entityId":"%s"%teacherOsid,
    "name":"teacherAcademicsQualification",
    "propertiesOSID": {
        "academicQualifications": [qualificationOsid]
    }
}
response = requests.put(baseUrl+'send', headers=headers, json=data)
print(response.status_code)
response.json()
assert response.status_code == 200
print(response.json())
assert response.json()["params"]["status"] == "SUCCESSFUL"



200
{'id': 'sunbird-rc.registry.send', 'ver': '1.0', 'ets': 1647347020325, 'params': {'resmsgid': '', 'msgid': '78400fec-8639-40f3-8389-3f9cf5b22a49', 'err': '', 'status': 'SUCCESSFUL', 'errmsg': ''}, 'responseCode': 'OK', 'result': {'attestationOSID': '1-884812fd-e8e9-4cda-b414-1f7dfce4d285'}}


In [116]:

# get institute token
institute1Token = getToken(institute1)
print(institute1Token)

# get institute claims and grant the claim
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'bearer %s'%institute1Token,
}

response = requests.request("GET", baseUrl + "Institute/claims", headers=headers)
assert response.status_code == 200
print(response)

# Attesting the claim as GRANT
data = {
    "action":"GRANT_CLAIM",
}
print(response.json())
attestingClaimId = None
for claim in response.json():
#     print(claim)
    print(claim["status"])
    if claim["status"] == "OPEN":
        attestingClaimId = claim["id"]
        print(claim["id"])
        attestResp = requests.request("POST", "%sInstitute/claims/%s/attest" %(baseUrl, claim["id"]), headers=headers, json=data)
        assert response.status_code == 200
        print(attestResp)
        print(attestResp.json())
        break
assert attestingClaimId != None

eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJtRTZ0WlFxSWctWHhGaVJPR21ERXZOb2ctR2Y4RklkNkNTaEJqMVQ2Qkc0In0.eyJleHAiOjE2NDczNDc2MjQsImlhdCI6MTY0NzM0NzAyNCwianRpIjoiMzQ5Mzk5YWItZjkxNi00YWNjLWIyZmUtOGQyOWI5MGZkOWY2IiwiaXNzIjoiaHR0cDovL2tleWNsb2FrOjgwODAvYXV0aC9yZWFsbXMvc3VuYmlyZC1yYyIsImF1ZCI6ImFjY291bnQiLCJzdWIiOiJhNDM3ZDQwMi1lNWFiLTQ3M2QtYWZjZi04MDZhMDBjNjY1MzIiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJyZWdpc3RyeS1mcm9udGVuZCIsInNlc3Npb25fc3RhdGUiOiI0ZWM1Y2NkMC02NDQxLTRjYzctYmZiOS05MTE2YjE2ZjlkMzYiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwczovL2xvY2FsaG9zdDo0MjAwIiwiaHR0cHM6Ly9uZGVhci54aXYuaW4iLCJodHRwOi8vbG9jYWxob3N0OjQyMDAiLCJodHRwOi8vbmRlYXIueGl2LmluIiwiaHR0cDovLzIwLjE5OC42NC4xMjgiXSwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iLCJkZWZhdWx0LXJvbGVzLW5kZWFyIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19

In [117]:

import time
time.sleep(3)
# verify attestation status
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'bearer %s'%teacherToken,
}

response = requests.request("GET", baseUrl + "Teacher", headers=headers)


for affiliation in response.json()[0]["teacherAcademicsQualification"]:
    print(affiliation)
    if affiliation["_osClaimId"] == attestingClaimId:
        assert affiliation["_osState"] == "PUBLISHED"

{'osUpdatedAt': '2022-03-15T12:23:44.788292Z', 'osUpdatedBy': '', '_osState': 'PUBLISHED', 'entityId': '1-eff29030-00a3-48cf-877f-a13244ed1fc2', 'osid': '1-884812fd-e8e9-4cda-b414-1f7dfce4d285', 'propertiesOSID': {'osid': '1-7c8b2cfd-a567-4716-90b7-2493fe648a8c', 'osUpdatedAt': '2022-03-15T12:23:40.550652Z', 'academicQualifications': ['1-ad81ff95-eb57-4b90-998c-5306a3251b7d'], 'osUpdatedBy': '4fbe8005-167d-4296-87c1-053f7eb7b9e7'}, 'propertyData': '{"academicQualifications":[{"osUpdatedAt":"2022-03-15T12:23:29.170474Z","osCreatedAt":"2022-03-15T12:23:18.989017Z","osUpdatedBy":"","osCreatedBy":"","instituteOSID":"1-0e343abd-c710-45a3-a240-e5d8f30ae2d2","osid":"1-ad81ff95-eb57-4b90-998c-5306a3251b7d","instituteName":"ABC"}]}', 'entityName': 'Teacher', 'name': 'teacherAcademicsQualification', '_osAttestedData': '{"academicQualifications":[{"osUpdatedAt":"2022-03-15T12:23:29.170474Z","osCreatedAt":"2022-03-15T12:23:18.989017Z","osUpdatedBy":"","osCreatedBy":"","instituteOSID":"1-0e343abd-c

# Student Attestations

In [150]:
# create teacher
studentUserId = "s4"
studentRequestBody = {
  "identityDetails": {
    "fullName": "student1",
    "gender": "Male",
    "dob": "1995-10-28",
    "identityHolder": {
      "AADHAAR": "123412323"
    }
  },
  "contactDetails": {
    "email": "%s@g.com"%studentUserId,
    "mobile": studentUserId
  },
  "educationDetails": [
    {
      "instituteOSID": "1-a140c49c-3575-471c-9e6c-e079fe920349",
      "instituteName": "XYZ"
    }
  ]
}
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", baseUrl + "Student/invite", headers=headers, json=studentRequestBody)

print(response.text)
assert response.status_code == 200
studentOsid = response.json()["result"]["Student"]["osid"]

{"id":"sunbird-rc.registry.invite","ver":"1.0","ets":1647350487384,"params":{"resmsgid":"","msgid":"7bb40e51-f057-4662-baa1-1c2a521d0b75","err":"","status":"SUCCESSFUL","errmsg":""},"responseCode":"OK","result":{"Student":{"osid":"1-d77f3eba-16ba-4aff-bde5-5eda1cda32db"}}}


In [151]:
studentToken = getToken(studentUserId)
print(studentToken)

eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJtRTZ0WlFxSWctWHhGaVJPR21ERXZOb2ctR2Y4RklkNkNTaEJqMVQ2Qkc0In0.eyJleHAiOjE2NDczNTEwODgsImlhdCI6MTY0NzM1MDQ4OCwianRpIjoiNzM4YTdlMzktZTk1Yy00ZjQ0LWIxODYtMDg1ODJhNTljOWZjIiwiaXNzIjoiaHR0cDovL2tleWNsb2FrOjgwODAvYXV0aC9yZWFsbXMvc3VuYmlyZC1yYyIsImF1ZCI6ImFjY291bnQiLCJzdWIiOiI3YjNiODYzMy1kNDhhLTRiOWQtOGZiYi1hYjk2ZDdlNGQxOWUiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJyZWdpc3RyeS1mcm9udGVuZCIsInNlc3Npb25fc3RhdGUiOiIxYjA0YTU5MC1mMGE3LTQ0MmEtODdlNS00N2ZmZTg4ZWUwZTIiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwczovL2xvY2FsaG9zdDo0MjAwIiwiaHR0cHM6Ly9uZGVhci54aXYuaW4iLCJodHRwOi8vbG9jYWxob3N0OjQyMDAiLCJodHRwOi8vbmRlYXIueGl2LmluIiwiaHR0cDovLzIwLjE5OC42NC4xMjgiXSwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iLCJkZWZhdWx0LXJvbGVzLW5kZWFyIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19

In [152]:
# get student details
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'bearer %s'%studentToken,
}

response = requests.request("GET", baseUrl + "Student", headers=headers)

print(response.text)
assert response.status_code == 200
print(response.json()[0]["educationDetails"][0]["osid"])
educationDetailsOsid = response.json()[0]["educationDetails"][0]["osid"]

[{"educationDetails":[{"osid":"1-914fdd90-0cf1-477b-ae5e-f1c9a1545667","instituteOSID":"1-a140c49c-3575-471c-9e6c-e079fe920349","instituteName":"XYZ"}],"contactDetails":{"osid":"1-f3768c65-c6bd-4d22-9440-f328b47ce8ef","mobile":"s4","email":"s4@g.com"},"osid":"1-d77f3eba-16ba-4aff-bde5-5eda1cda32db","osOwner":["7b3b8633-d48a-4b9d-8fbb-ab96d7e4d19e"],"identityDetails":{"osid":"1-6985aa68-0785-4b6d-be0d-5cfa0210d7c6","identityHolder":{"osid":"1-a3695ffe-2170-4510-956c-ba1f643c61bf","AADHAAR":"123412323"},"gender":"Male","dob":"1995-10-28","fullName":"student1"}}]
1-914fdd90-0cf1-477b-ae5e-f1c9a1545667


In [153]:
# raise qualification attestation request
headers = {
    'content-type': 'application/json',
    'authorization': 'bearer %s'%studentToken,
}

data = {
    "entityName":"Student",
    "entityId":"%s"%studentOsid,
    "name":"studentInstituteAttest",
    "propertiesOSID": {
        "educationDetails": [educationDetailsOsid]
    }
}
response = requests.put(baseUrl+'send', headers=headers, json=data)
print(response.status_code)
response.json()
assert response.status_code == 200
print(response.json())
assert response.json()["params"]["status"] == "SUCCESSFUL"


200
{'id': 'sunbird-rc.registry.send', 'ver': '1.0', 'ets': 1647350493478, 'params': {'resmsgid': '', 'msgid': '7a27f91f-741e-40a8-b50e-c6aec1310466', 'err': '', 'status': 'SUCCESSFUL', 'errmsg': ''}, 'responseCode': 'OK', 'result': {'attestationOSID': '1-b8095c70-b406-471a-b22d-8356e71c686c'}}


In [154]:
# get teacher token
teacherToken = getToken(teacherUserId)
print(teacherToken)

# get institute claims and grant the claim
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'bearer %s'%teacherToken,
}

response = requests.request("GET", baseUrl + "Teacher/claims", headers=headers)
assert response.status_code == 200
print(response)

# Attesting the claim as GRANT
data = {
    "action":"GRANT_CLAIM",
}
print(response.json())
attestingClaimId = None
for claim in response.json():
#     print(claim)
    print(claim["status"])
    if claim["status"] == "OPEN":
        attestingClaimId = claim["id"]
        print(claim["id"])
        attestResp = requests.request("POST", "%sTeacher/claims/%s/attest" %(baseUrl, claim["id"]), headers=headers, json=data)
        assert response.status_code == 200
        print(attestResp)
        print(attestResp.json())
        break
assert attestingClaimId != None

eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJtRTZ0WlFxSWctWHhGaVJPR21ERXZOb2ctR2Y4RklkNkNTaEJqMVQ2Qkc0In0.eyJleHAiOjE2NDczNTEwOTYsImlhdCI6MTY0NzM1MDQ5NiwianRpIjoiYTBlZjQxMDItMTNkYy00ZTZkLThiNTAtNzNiOTJjYmYzYmY0IiwiaXNzIjoiaHR0cDovL2tleWNsb2FrOjgwODAvYXV0aC9yZWFsbXMvc3VuYmlyZC1yYyIsImF1ZCI6ImFjY291bnQiLCJzdWIiOiI0ZmJlODAwNS0xNjdkLTQyOTYtODdjMS0wNTNmN2ViN2I5ZTciLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJyZWdpc3RyeS1mcm9udGVuZCIsInNlc3Npb25fc3RhdGUiOiI0ZGY0YjFlNy0zNTk1LTQ4OWMtODlkNy1iZjc5ZGFkODZhMTgiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwczovL2xvY2FsaG9zdDo0MjAwIiwiaHR0cHM6Ly9uZGVhci54aXYuaW4iLCJodHRwOi8vbG9jYWxob3N0OjQyMDAiLCJodHRwOi8vbmRlYXIueGl2LmluIiwiaHR0cDovLzIwLjE5OC42NC4xMjgiXSwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iLCJkZWZhdWx0LXJvbGVzLW5kZWFyIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19

In [155]:

import time
time.sleep(1)
# verify attestation status
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'bearer %s'%studentToken,
}

response = requests.request("GET", baseUrl + "Student", headers=headers)


for affiliation in response.json()[0]["studentInstituteAttest"]:
    print(affiliation)
    if affiliation["_osClaimId"] == attestingClaimId:
        assert affiliation["_osState"] == "PUBLISHED"

{'entityName': 'Student', '_osState': 'PUBLISHED', 'name': 'studentInstituteAttest', 'entityId': '1-d77f3eba-16ba-4aff-bde5-5eda1cda32db', 'osid': '1-b8095c70-b406-471a-b22d-8356e71c686c', '_osAttestedData': '{"name":"student1","educationDetails":[{"osid":"1-914fdd90-0cf1-477b-ae5e-f1c9a1545667","instituteOSID":"1-a140c49c-3575-471c-9e6c-e079fe920349","instituteName":"XYZ"}]}', 'propertiesOSID': {'osid': '1-709482b4-2f91-454a-ae22-d5ccd73fe7e1', 'educationDetails': ['1-914fdd90-0cf1-477b-ae5e-f1c9a1545667']}, '_osClaimId': '9aed418a-b45a-4c3f-840e-9da4ebddf613', 'propertyData': '{"name":"student1","educationDetails":[{"osid":"1-914fdd90-0cf1-477b-ae5e-f1c9a1545667","instituteOSID":"1-a140c49c-3575-471c-9e6c-e079fe920349","instituteName":"XYZ"}]}'}
